In [2]:
import re
from collections import defaultdict

from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

In [3]:
from operator import iadd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy._compat import to_unicode
from sumy.summarizers.edmundson import EdmundsonSummarizer

import pandas as pd
import numpy as np
import math
import time

import string

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/aryan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
cd wcep-mds-dataset/experiments

/home/aryan/Documents/2-1/CL 2/Project/Midsem_Team 1/wcep-mds-dataset/experiments


In [5]:
import utils
from rouge_score import rouge_scorer
 
dataset = list(utils.read_jsonl_gz('WCEP/val.jsonl.gz'))
val_data = list(utils.read_jsonl_gz('WCEP/val.jsonl.gz'))
print(val_data[0].keys())


dict_keys(['id', 'date', 'reference_urls', 'articles', 'summary', 'wiki_links', 'category'])


In [6]:
def evaluate(summary, gold_summary):
    scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
    scores = scorer.score(" ".join(summary),gold_summary)
    
    with open("scores.txt", "a") as f:
        f.write(f"f-measure = {scores['rouge1']}\n")

In [7]:
def writeSummary(summary, golden_summary, title):
    with open("articles.txt", "a") as f:
        with open('summaries.txt', "a") as f:
            f.write(f"{title}\n{summary}\nGolden summary:\n{golden_summary}\n\n")

In [9]:
no_of_clusters = 20 # as many as you want
ps = PorterStemmer()
for i in range(no_of_clusters):
    cluster = dataset[i]
    golden_summary = cluster['summary']
    articles = cluster['articles']
    text = articles[0]['text'] # you can take any index you want
    title = articles[0]['title']

    with open("articles.txt", "a") as f:
        f.write(text) 
# you'll have to create a file articles.txt and make sure that it's empty every time you run this script

    data = text
    final_sent_list = sent_tokenize(text)

    data= re.sub(r"\n", " ", data)
    data= re.sub("\""," ",data)
    data = re.sub("''"," ",data)
    data = re.sub("``"," ",data)
    data = re.sub(" +"," ",data)
    #data = re.sub(r'[^a-zA-Z0-9\s]', ' ', data)

    sentences = sent_tokenize(data)

    cleaned_sentences = []
    for x in sentences:
        a = x.lower()
        cleaned_sentences.append(a)

    matrix = []
    sentences = sent_tokenize(data)
    total_Sent = len(cleaned_sentences)

    for sent in cleaned_sentences:
        words = word_tokenize(sent)
        matrix.append(words)
    
    matrix_die = []
    emb = []
    for i in matrix:
        for j in i:

            a = ps.stem(j)
            emb.append(a)
        matrix_die.append(emb)
        emb = []

    #1)stigma words
    file = "stigma.txt"
    with open(file, 'r') as f:
        stigma = f.read()

    stig = word_tokenize(stigma)
    stig = sorted(stig)

    stigma_list = []
    for word in stig:
        word = word.lower()
        stigma_list.append(ps.stem(word))

    #2)bonus words
    file = "bonus.txt"
    with open(file, 'r') as f:
        bonus = f.read()

    bon = word_tokenize(bonus)
    bon = sorted(bon)

    bonus_list = []
    for word in bon:
        word = word.lower()
        bonus_list.append(ps.stem(word))

    #3)null words
    file = "null.txt"
    with open(file, 'r') as f:
        null_ = f.read()

    nul = word_tokenize(null_)
    nul = sorted(nul)

    null_list = []
    for word in nul:
        word = word.lower()
        null_list.append(ps.stem(word)) 
   
    weights_cue = {}
    #initializing weight of all sentences to 0
    weights_cue = {sent_weight: 0 for sent_weight in range(total_Sent)}

    for i in matrix_die:
        ans = matrix_die.index(i)
        for j in i:
            if j in bonus_list:
                #print(weights_cue)
                weights_cue[ans] = weights_cue[ans] + 1.5
            elif j in stigma_list:
                weights_cue[ans] = weights_cue[ans] + 0.5
            elif j in null_list:
                weights_cue[ans] = weights_cue[ans] + 1


    stop_words = stopwords.words("english")
    stems = []

    for sent in matrix_die:
        for word in sent:
            if word not in (stop_words) and (bonus_list) and (stigma_list) and (null_list):
                stems.append(word)

    #removing punctuation
    punctuation = ['.',',','"','!',':','"','”', '“', '(',')','[',']','{','}',':',';','/','+','-','=','_','e.g',"'",'’']
    new_words = []

    for word in list(stems):  # iterating on a copy since removing will mess things up
        if word in (punctuation):# or (bonus_list): #or (stigma_list) or (null_list):
            stems.remove(word)

    # Word frequencies
    frequency_table = dict()
    for word in stems:
        if word in stop_words:
            continue
        if word in frequency_table:
            frequency_table[word] += 1
        else:
            frequency_table[word] = 1

#create dictionary for 
    Keymax = max(zip(frequency_table.values(), frequency_table.keys()))[1]
#print(Keymax)
#set threshold over which we consider frequencies
    thresh = 6
    weights_key = {}
#initializing weight of all sentences to 0
    weights_key = {sent_weight: 0 for sent_weight in range(total_Sent)}
#print(type(weights_key))

# for i in stems:
#     ans = stems.index(i)
#     if frequency_table[i] > thresh:
#         weights_key[ans] = weights_key[ans] + frequency_table[i]

    for sent in matrix_die:
        ans = matrix_die.index(sent)
        for word in sent:
            if word in frequency_table:
                #ind = frequency_table.index(word)
                if frequency_table[word] > thresh:
                    weights_key[ans] = weights_key[ans] + frequency_table[word]


    #dictionary for key method sentence weights:
    weights_title = {}
    #initializing weight of all sentences to 0
    weights_title = {sent_weight: 0 for sent_weight in range(total_Sent)}

#getting list of title, heading words
#we can make two lists one for title and one for heading.
#heading list weight > title list weight
#these lists are user input

#In this case, we made our list of (only) heading words for our dataset
#This is because our dataset is a story. so it only has headings, no titles.
    #file = "heading.txt"
    with open("heading.txt", 'r') as f:
        head = f.read()

    he = word_tokenize(head)
    he = sorted(he)

    heading_list = []
    title_list = [] #not using this for stories, only technical docs
    for word in he:
        word = word.lower()
        heading_list.append(ps.stem(word))

    for i in matrix_die:
        ans = matrix_die.index(i)
        for j in i:
            if j in heading_list:
            #print(weights_cue)
                weights_title[ans] = weights_title[ans] + 3
            elif j in title_list:
                weights_title[ans] = weights_title[ans] + 2

#According to Edmundson, sentences at the beginning of a document are more relevant than the end. 
#The sentence position score metric used in this algorithm was introduced by Joshi in 2019.

#centroid = sum/len(cleaned_sentences)

#dictionary for location method sentence weights:
    weights_location = {}
    weights_location = {sent_weight: 0 for sent_weight in range(total_Sent)}

    for i,sent in enumerate(cleaned_sentences):
        weights_location[i] = max(0.5,math.exp(-(i+1)/(len(cleaned_sentences)**(1/3))))


#dictionary to sum up all 4 weights
    weights = {}

#user input prioritization for the 4 weighing methods
    ALPHA = 4
    BETA = 8
    GAMMA = 1
    OMEGA = 10

    for sent in cleaned_sentences:
        i = cleaned_sentences.index(sent)
        weights[i] = (ALPHA*weights_cue[i])+(BETA*weights_key[i])+(GAMMA*weights_title[i])+(OMEGA*weights_location[i])

    sorted_weights = {}
    sorted_weights = dict(sorted(weights.items(), key=lambda item: item[1], reverse=True))

    summary = ""
    i = []
# for sent in sorted_weights:
#     i = sorted_weights.keys()s

# print(i)
    keys = sorted_weights.keys()
    keys = list(keys)
    for sent in keys:
        if len(summary)< 1500: #len(word_tokenize(text))//2:   #making summary of approx 50% length
            summary = summary + final_sent_list[sent]

    print(summary)

    evaluate(summary, golden_summary)
    writeSummary(summary, golden_summary, title)

    # with open("article.txt",'w') as file:
    #     pass

We have torn off the last chains that tied us to Moscow with its fantasies of Ukraine as a canonical territory of the Russian Orthodox Church," Poroshenko said.The service came a day after the leader of the world's Orthodox, Ecumenical Partriarch Bartholomew I, presented a decree recognizing the independent Orthodox Church of Ukraine.KIEV, Ukraine (AP) — Ukrainian President Petro Poroshenko says the establishment of an Orthodox Church independent of the Russian church is a major step toward moving out of Russia's shadow.Previously, the Ukrainian church that was tied to the Moscow Patriarchate was considered canonical, or legitimate.On Christmas Day, which Ukrainian Orthodox believers celebrated on Monday, Poroshenko attended services in the capital's Sophia Cathedral."This is the foundation of our spiritual freedom.
KABUL: At least 30 Afghans were killed on Sunday (Jan 6) when the tunnel they were digging in to mine gold in northern Afghanistan collapsed, officials said.Rohani said abo